In [88]:
from cassandra.cluster import Cluster
import time

## Initialization

In [2]:
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()
print(session.execute('SELECT release_version FROM system.local').one())

Row(release_version='3.11.11')


In [3]:
session.execute("CREATE KEYSPACE IF NOT EXISTS estore WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor' : '1' };")

rows = session.execute("SELECT * FROM system_schema.keyspaces;").all()
for row in rows:
    print(row)

Row(keyspace_name='system_auth', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '1')]))
Row(keyspace_name='system_schema', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.LocalStrategy')]))
Row(keyspace_name='estore', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '1')]))
Row(keyspace_name='system_distributed', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')]))
Row(keyspace_name='system', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.LocalStrategy')]))
Row(keyspace_name='system_traces', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replicat

## Items table

In [4]:
session.execute("DROP MATERIALIZED VIEW IF EXISTS estore.items_by_price;")
session.execute("DROP MATERIALIZED VIEW IF EXISTS estore.items_by_producer_price;")
session.execute("DROP MATERIALIZED VIEW IF EXISTS estore.items_by_name;")
session.execute("DROP TABLE IF EXISTS estore.items;")

query = """
CREATE TABLE estore.items (
category text,
producer text,
item_id int,
name text,
price int,
production_year int,
options map<text,text>,
PRIMARY KEY (category, producer, item_id)
);
"""
session.execute(query)


In [5]:
query = """
CREATE INDEX IF NOT EXISTS options_idx
ON estore.items ( ENTRIES (options) );
"""

session.execute(query)

In [6]:

query = """
CREATE INDEX IF NOT EXISTS option_keys_idx
ON estore.items ( KEYS (options) );
"""

session.execute(query)

In [7]:
query = """
CREATE MATERIALIZED VIEW IF NOT EXISTS estore.items_by_price AS
SELECT category, price, producer, item_id, name, production_year, options
FROM estore.items
WHERE category IS NOT NULL AND price IS NOT NULL AND producer IS NOT NULL AND item_id is NOT NULL
PRIMARY KEY (category, price, producer, item_id)
WITH CLUSTERING ORDER BY (price ASC);
"""
session.execute(query)

In [8]:
query = """
CREATE MATERIALIZED VIEW IF NOT EXISTS estore.items_by_producer_price AS
SELECT category, price, producer, item_id, name, production_year, options
FROM estore.items
WHERE category IS NOT NULL AND producer IS NOT NULL AND price IS NOT NULL AND item_id is NOT NULL
PRIMARY KEY (category, producer, price, item_id)
WITH CLUSTERING ORDER BY (price ASC);
"""

session.execute(query)

In [9]:
query = """
CREATE MATERIALIZED VIEW IF NOT EXISTS estore.items_by_name AS
SELECT category, price, producer, item_id, name, production_year, options
FROM estore.items
WHERE category IS NOT NULL AND name IS NOT NULL AND producer IS NOT NULL AND item_id is NOT NULL
PRIMARY KEY (category, name, producer, item_id);
"""

session.execute(query)

### Inserting some data to items table

In [10]:
query="""
BEGIN BATCH
    INSERT INTO estore.items (category, producer, item_id, name, price, production_year, options)
    VALUES ('Smartphones', 'Apple', 1, 'Ipnone 11', 18499, 2020, {'memory' : '64Gb'}) IF NOT EXISTS;
    INSERT INTO estore.items (category, producer, item_id, name, price, production_year, options)
    VALUES ('Smartphones', 'Apple', 2, 'Ipnone 13', 29999, 2021, {}) IF NOT EXISTS;
    INSERT INTO estore.items (category, producer, item_id, name, price, production_year, options)
    VALUES ('Smartphones', 'Apple', 3, 'Ipnone 13 Pro', 37499, 2021, {}) IF NOT EXISTS;
    INSERT INTO estore.items (category, producer, item_id, name, price, production_year, options)
    VALUES ('Smartphones', 'Apple', 4, 'Ipnone 13 Pro Max', 41499, 2021, {'memory' : '128Gb'}) IF NOT EXISTS;
    INSERT INTO estore.items (category, producer, item_id, name, price, production_year, options)
    VALUES ('Smartphones', 'Samsung', 5, 'Galaxy S21 FE', 21999, 2022, {'memory' : '64Gb'}) IF NOT EXISTS;
    INSERT INTO estore.items (category, producer, item_id, name, price, production_year, options)
    VALUES ('Smartphones', 'Samsung', 6, 'Galaxy S21', 22999, 2021, {'memory' : '128Gb'}) IF NOT EXISTS;
    INSERT INTO estore.items (category, producer, item_id, name, price, production_year, options)
    VALUES ('Smartphones', 'Samsung', 7, 'Galaxy S21 Plus', 23999, 2021, {}) IF NOT EXISTS;
    INSERT INTO estore.items (category, producer, item_id, name, price, production_year, options)
    VALUES ('Smartphones', 'Samsung', 8, 'Galaxy S21 Ultra', 39999, 2021, {}) IF NOT EXISTS;    
APPLY BATCH;
"""
session.execute(query)



In [11]:
query="""
BEGIN BATCH
    INSERT INTO estore.items (category, producer, item_id, name, price, production_year, options)
    VALUES ('Wearables', 'Apple', 9, 'Apple Watch Series 7', 13999, 2021, {'connectivity' : 'WiFi', 'type' : 'smartwatch'}) IF NOT EXISTS;
    INSERT INTO estore.items (category, producer, item_id, name, price, production_year, options)
    VALUES ('Wearables', 'Samsung', 10, 'Galaxy Watch 4 Classic', 11499, 2021, {'connectivity' : 'LTE', 'type' : 'smartwatch'}) IF NOT EXISTS;  
APPLY BATCH;
"""
session.execute(query)

In [12]:
query="""
BEGIN BATCH
    INSERT INTO estore.items (category, producer, item_id, name, price, production_year, options)
    VALUES ('Tablets', 'Samsung', 11, 'Galaxy Tab S7', 23999, 2021, {'memory' : '128Gb', 'display': '11.4in'}) IF NOT EXISTS;
    INSERT INTO estore.items (category, producer, item_id, name, price, production_year, options)
    VALUES ('Tablets', 'Apple', 12, 'iPad Pro', 41999, 2021, {'memory' : '128Gb', 'display': '12.9in'}) IF NOT EXISTS;
APPLY BATCH;
"""
session.execute(query)

In [13]:

query="""
BEGIN BATCH
    INSERT INTO estore.items (category, producer, item_id, name, price, production_year, options)
    VALUES ('TV', 'Samsung', 13, 'Samsung QE55', 30999, 2021, {'technology': 'QLED'}) IF NOT EXISTS;
    INSERT INTO estore.items (category, producer, item_id, name, price, production_year, options)
    VALUES ('TV', 'Hisense', 14, 'Hisense 55E', 19999, 2021, {'technology': 'IPS'}) IF NOT EXISTS;
    INSERT INTO estore.items (category, producer, item_id, name, price, production_year, options)
    VALUES ('TV', 'LG', 15, 'LG OLED55', 41999, 2021, {'technology': 'OLED'}) IF NOT EXISTS;
    INSERT INTO estore.items (category, producer, item_id, name, price, production_year, options)
    VALUES ('TV', 'Philips', 16, 'Philips 55OLED', 45999, 2021, {'technology': 'OLED'}) IF NOT EXISTS;
APPLY BATCH;
"""
session.execute(query)
    
    
    
    

### Querring items table

In [55]:
# 1. Показати структуру таблиці

session.execute("DESCRIBE estore.items;")

SyntaxException: <Error from server: code=2000 [Syntax error in CQL query] message="line 1:0 no viable alternative at input 'DESCRIBE' ([DESCRIBE]...)">

##### DESCRIBE directive is not supported by python's driver. 
##### Issue addressed here: https://stackoverflow.com/questions/35986136/cassandra-3-1-python-driver-no-viable-alternative-at-input-describe
##### Since that, it's output is got by entering cassandra's container shell end executing it with cqlsh 

    cqlsh> DESCRIBE estore.items;

    CREATE TABLE estore.items (
        category text,
        producer text,
        item_id int,
        name text,
        options map<text, text>,
        price int,
        production_year int,
        PRIMARY KEY (category, producer, item_id)
    ) WITH CLUSTERING ORDER BY (producer ASC, item_id ASC)
        AND bloom_filter_fp_chance = 0.01
        AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
        AND comment = ''
        AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
        AND compression = {'chunk_length_in_kb': '64', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
        AND crc_check_chance = 1.0
        AND dclocal_read_repair_chance = 0.1
        AND default_time_to_live = 0
        AND gc_grace_seconds = 864000
        AND max_index_interval = 2048
        AND memtable_flush_period_in_ms = 0
        AND min_index_interval = 128
        AND read_repair_chance = 0.0
        AND speculative_retry = '99PERCENTILE';
    CREATE INDEX option_keys_idx ON estore.items (keys(options));
    CREATE INDEX options_idx ON estore.items (entries(options));

    CREATE MATERIALIZED VIEW estore.items_by_price AS
        SELECT category, price, producer, item_id, name, options, production_year
        FROM estore.items
        WHERE category IS NOT NULL AND price IS NOT NULL AND producer IS NOT NULL AND item_id IS NOT NULL
        PRIMARY KEY (category, price, producer, item_id)
        WITH CLUSTERING ORDER BY (price ASC, producer ASC, item_id ASC)
        AND bloom_filter_fp_chance = 0.01
        AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
        AND comment = ''
        AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
        AND compression = {'chunk_length_in_kb': '64', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
        AND crc_check_chance = 1.0
        AND dclocal_read_repair_chance = 0.1
        AND default_time_to_live = 0
        AND gc_grace_seconds = 864000
        AND max_index_interval = 2048
        AND memtable_flush_period_in_ms = 0
        AND min_index_interval = 128
        AND read_repair_chance = 0.0
        AND speculative_retry = '99PERCENTILE';

    CREATE MATERIALIZED VIEW estore.items_by_producer_price AS
        SELECT category, producer, price, item_id, name, options, production_year
        FROM estore.items
        WHERE category IS NOT NULL AND producer IS NOT NULL AND price IS NOT NULL AND item_id IS NOT NULL
        PRIMARY KEY (category, producer, price, item_id)
        WITH CLUSTERING ORDER BY (producer ASC, price ASC, item_id ASC)
        AND bloom_filter_fp_chance = 0.01
        AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
        AND comment = ''
        AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
        AND compression = {'chunk_length_in_kb': '64', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
        AND crc_check_chance = 1.0
        AND dclocal_read_repair_chance = 0.1
        AND default_time_to_live = 0
        AND gc_grace_seconds = 864000
        AND max_index_interval = 2048
        AND memtable_flush_period_in_ms = 0
        AND min_index_interval = 128
        AND read_repair_chance = 0.0
        AND speculative_retry = '99PERCENTILE';

    CREATE MATERIALIZED VIEW estore.items_by_name AS
        SELECT category, name, producer, item_id, options, price, production_year
        FROM estore.items
        WHERE category IS NOT NULL AND name IS NOT NULL AND producer IS NOT NULL AND item_id IS NOT NULL
        PRIMARY KEY (category, name, producer, item_id)
        WITH CLUSTERING ORDER BY (name ASC, producer ASC, item_id ASC)
        AND bloom_filter_fp_chance = 0.01
        AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
        AND comment = ''
        AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
        AND compression = {'chunk_length_in_kb': '64', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
        AND crc_check_chance = 1.0
        AND dclocal_read_repair_chance = 0.1
        AND default_time_to_live = 0
        AND gc_grace_seconds = 864000
        AND max_index_interval = 2048
        AND memtable_flush_period_in_ms = 0
        AND min_index_interval = 128
        AND read_repair_chance = 0.0
        AND speculative_retry = '99PERCENTILE';

In [15]:
# 2. Напишіть запит, який виводить усі товари в певній категорії відсортовані за ціною

query = """
SELECT * FROM estore.items_by_price
WHERE category = 'TV'
"""

rows = session.execute(query)
for row in rows:
    print(row)

Row(category='TV', price=19999, producer='Hisense', item_id=14, name='Hisense 55E', options=OrderedMapSerializedKey([('technology', 'IPS')]), production_year=2021)
Row(category='TV', price=30999, producer='Samsung', item_id=13, name='Samsung QE55', options=OrderedMapSerializedKey([('technology', 'QLED')]), production_year=2021)
Row(category='TV', price=41999, producer='LG', item_id=15, name='LG OLED55', options=OrderedMapSerializedKey([('technology', 'OLED')]), production_year=2021)
Row(category='TV', price=45999, producer='Philips', item_id=16, name='Philips 55OLED', options=OrderedMapSerializedKey([('technology', 'OLED')]), production_year=2021)


In [16]:
# 3.1. Напишіть запити, які вибирають товари за різними критеріями в межах певної категорії 
# (тут де треба замість індексу використайте Matirialized view): назва

query = """
SELECT * FROM estore.items_by_name
WHERE category = 'Smartphones'
AND name = 'Ipnone 11'
"""

rows = session.execute(query)
for row in rows:
    print(row)

Row(category='Smartphones', name='Ipnone 11', producer='Apple', item_id=1, options=OrderedMapSerializedKey([('memory', '64Gb')]), price=18499, production_year=2020)


In [17]:
# 3.2. Напишіть запити, які вибирають товари за різними критеріями в межах певної категорії 
# (тут де треба замість індексу використайте Matirialized view): ціна (в проміжку)

query = """
SELECT * FROM estore.items_by_price
WHERE category = 'Smartphones'
AND price >= 20000 AND price <= 25000
"""

rows = session.execute(query)
for row in rows:
    print(row)

Row(category='Smartphones', price=21999, producer='Samsung', item_id=5, name='Galaxy S21 FE', options=OrderedMapSerializedKey([('memory', '64Gb')]), production_year=2022)
Row(category='Smartphones', price=22999, producer='Samsung', item_id=6, name='Galaxy S21', options=OrderedMapSerializedKey([('memory', '128Gb')]), production_year=2021)
Row(category='Smartphones', price=23999, producer='Samsung', item_id=7, name='Galaxy S21 Plus', options=None, production_year=2021)


In [18]:
# 3.3. Напишіть запити, які вибирають товари за різними критеріями в межах певної категорії 
# (тут де треба замість індексу використайте Matirialized view): ціна та виробник

query = """
SELECT * FROM estore.items_by_producer_price
WHERE category = 'Smartphones'
AND producer = 'Apple'
AND price >= 15000 AND price <= 25000
"""

rows = session.execute(query)
for row in rows:
    print(row)

Row(category='Smartphones', producer='Apple', price=18499, item_id=1, name='Ipnone 11', options=OrderedMapSerializedKey([('memory', '64Gb')]), production_year=2020)


In [19]:
# 4.1. Напишіть запити, які вибирають товари за: наявність певних характеристик

query = """
SELECT * FROM estore.items
WHERE options CONTAINS KEY 'memory'
"""

rows = session.execute(query)
for row in rows:
    print(row)

Row(category='Tablets', producer='Apple', item_id=12, name='iPad Pro', options=OrderedMapSerializedKey([('display', '12.9in'), ('memory', '128Gb')]), price=41999, production_year=2021)
Row(category='Tablets', producer='Samsung', item_id=11, name='Galaxy Tab S7', options=OrderedMapSerializedKey([('display', '11.4in'), ('memory', '128Gb')]), price=23999, production_year=2021)
Row(category='Smartphones', producer='Apple', item_id=1, name='Ipnone 11', options=OrderedMapSerializedKey([('memory', '64Gb')]), price=18499, production_year=2020)
Row(category='Smartphones', producer='Apple', item_id=4, name='Ipnone 13 Pro Max', options=OrderedMapSerializedKey([('memory', '128Gb')]), price=41499, production_year=2021)
Row(category='Smartphones', producer='Samsung', item_id=5, name='Galaxy S21 FE', options=OrderedMapSerializedKey([('memory', '64Gb')]), price=21999, production_year=2022)
Row(category='Smartphones', producer='Samsung', item_id=6, name='Galaxy S21', options=OrderedMapSerializedKey([('

In [20]:
# 4.2. Напишіть запити, які вибирають товари за: певна характеристика та її значення

query = """
SELECT * FROM estore.items
WHERE options['display'] = '11.4in'
"""

rows = session.execute(query)
for row in rows:
    print(row)

Row(category='Tablets', producer='Samsung', item_id=11, name='Galaxy Tab S7', options=OrderedMapSerializedKey([('display', '11.4in'), ('memory', '128Gb')]), price=23999, production_year=2021)


In [26]:
# 5.1. Оновити опис товару: змініть існуючі значення певної характеристики

query = """
UPDATE estore.items
SET options['connectivity'] = 'LTE'
WHERE category = 'Smartphones' AND producer = 'Apple' AND item_id = 9
"""

session.execute(query)

query = """
SELECT * FROM estore.items
WHERE options['connectivity'] = 'LTE'
"""

rows = session.execute(query)
for row in rows:
    print(row)

Row(category='Smartphones', producer='Apple', item_id=9, name=None, options=OrderedMapSerializedKey([('connectivity', 'LTE')]), price=None, production_year=None)
Row(category='Wearables', producer='Samsung', item_id=10, name='Galaxy Watch 4 Classic', options=OrderedMapSerializedKey([('connectivity', 'LTE'), ('type', 'smartwatch')]), price=11499, production_year=2021)


In [30]:
# 5.2. Оновити опис товару: додайте нові властивості (характеристики) товару

query = """
UPDATE estore.items
SET options = options + {'display' : '55in'}
WHERE category = 'TV' AND producer = 'Hisense' AND item_id = 14
"""

session.execute(query)

query = """
SELECT * FROM estore.items
WHERE options['display'] = '55in'
"""

rows = session.execute(query)
for row in rows:
    print(row)

Row(category='TV', producer='Hisense', item_id=14, name='Hisense 55E', options=OrderedMapSerializedKey([('display', '55in'), ('technology', 'IPS')]), price=19999, production_year=2021)


In [31]:
# 5.3. Оновити опис товару: видалить характеристику товару

query = """
UPDATE estore.items
SET options = options - {'display'}
WHERE category = 'TV' AND producer = 'Hisense' AND item_id = 14
"""

session.execute(query)

query = """
SELECT * FROM estore.items
WHERE category = 'TV' AND producer = 'Hisense'
"""

rows = session.execute(query)
for row in rows:
    print(row)

Row(category='TV', producer='Hisense', item_id=14, name='Hisense 55E', options=OrderedMapSerializedKey([('technology', 'IPS')]), price=19999, production_year=2021)


## Orders table

In [60]:
session.execute("DROP MATERIALIZED VIEW IF EXISTS estore.orders_by_price;")
session.execute("DROP TABLE IF EXISTS estore.orders;")

query = """
CREATE TABLE estore.orders (
client_name text,
order_date date,
order_id int,
order_timestamp timestamp,
cost int,
items list<int>,
PRIMARY KEY (client_name, order_date, order_id)
);
"""

session.execute(query)


In [61]:
query = """
CREATE INDEX IF NOT EXISTS order_items_idx
ON estore.orders (items);
"""

session.execute(query)

In [62]:
query = """
CREATE MATERIALIZED VIEW IF NOT EXISTS estore.orders_by_price AS
SELECT client_name, order_date, order_id, order_timestamp, cost, items
FROM estore.orders
WHERE client_name IS NOT NULL AND order_date IS NOT NULL AND order_id IS NOT NULL AND cost is NOT NULL
PRIMARY KEY (client_name, cost, order_date, order_id)
WITH CLUSTERING ORDER BY (price ASC);
"""

session.execute(query)

### Inserting some data to orders table

In [63]:
query="""
BEGIN BATCH
    INSERT INTO estore.orders (client_name, order_date, order_id, order_timestamp, cost, items)
    VALUES ('Stepan Baran', '2022-01-10', 1, '2022-01-10 13:44', 51498, [3, 9]) IF NOT EXISTS;
    INSERT INTO estore.orders (client_name, order_date, order_id, order_timestamp, cost, items)
    VALUES ('Stepan Baran', '2022-01-13', 2, '2022-01-13 09:10', 41999, [15]) IF NOT EXISTS;
    INSERT INTO estore.orders (client_name, order_date, order_id, order_timestamp, cost, items)
    VALUES ('Stepan Baran', '2022-01-18', 3, '2022-01-18 18:35', 41999, [12]) IF NOT EXISTS;
APPLY BATCH;
"""

session.execute(query)

In [64]:
query="""
BEGIN BATCH
    INSERT INTO estore.orders (client_name, order_date, order_id, order_timestamp, cost, items)
    VALUES ('Petro Vovk', '2022-01-11', 4, '2022-01-11 13:23', 23999, [7]) IF NOT EXISTS;
    INSERT INTO estore.orders (client_name, order_date, order_id, order_timestamp, cost, items)
    VALUES ('Petro Vovk', '2022-01-12', 5, '2022-01-12 19:10', 35498, [10, 11]) IF NOT EXISTS;
    INSERT INTO estore.orders (client_name, order_date, order_id, order_timestamp, cost, items)
    VALUES ('Petro Vovk', '2022-01-19', 6, '2022-01-19 14:05', 30999, [13]) IF NOT EXISTS;
APPLY BATCH;
"""

session.execute(query)

In [65]:
query="""
BEGIN BATCH
    INSERT INTO estore.orders (client_name, order_date, order_id, order_timestamp, cost, items)
    VALUES ('Mykola Bez Prykolu', '2022-01-15', 7, '2022-01-15 13:23', 75497, [8, 10, 11]) IF NOT EXISTS;
    INSERT INTO estore.orders (client_name, order_date, order_id, order_timestamp, cost, items)
    VALUES ('Mykola Bez Prykolu', '2022-01-17', 8, '2022-01-17 19:10', 45999, [16]) IF NOT EXISTS;
APPLY BATCH;
"""

session.execute(query)

### Querying orders table

In [66]:
# 1. Показати структуру таблиці

session.execute("DESCRIBE estore.orders;")

SyntaxException: <Error from server: code=2000 [Syntax error in CQL query] message="line 1:0 no viable alternative at input 'DESCRIBE' ([DESCRIBE]...)">

##### DESCRIBE directive is not supported by python's driver. 
##### Issue addressed here: https://stackoverflow.com/questions/35986136/cassandra-3-1-python-driver-no-viable-alternative-at-input-describe
##### Since that, it's output is got by entering cassandra's container shell end executing it with cqlsh
    cqlsh> DESCRIBE estore.orders;

    CREATE TABLE estore.orders (
        client_name text,
        order_date date,
        order_id int,
        cost int,
        items list<int>,
        order_timestamp timestamp,
        PRIMARY KEY (client_name, order_date, order_id)
    ) WITH CLUSTERING ORDER BY (order_date ASC, order_id ASC)
        AND bloom_filter_fp_chance = 0.01
        AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
        AND comment = ''
        AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
        AND compression = {'chunk_length_in_kb': '64', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
        AND crc_check_chance = 1.0
        AND dclocal_read_repair_chance = 0.1
        AND default_time_to_live = 0
        AND gc_grace_seconds = 864000
        AND max_index_interval = 2048
        AND memtable_flush_period_in_ms = 0
        AND min_index_interval = 128
        AND read_repair_chance = 0.0
        AND speculative_retry = '99PERCENTILE';
    CREATE INDEX order_items_idx ON estore.orders (values(items));

    CREATE MATERIALIZED VIEW estore.orders_by_price AS
        SELECT client_name, cost, order_date, order_id, items, order_timestamp
        FROM estore.orders
        WHERE client_name IS NOT NULL AND order_date IS NOT NULL AND order_id IS NOT NULL AND cost IS NOT NULL
        PRIMARY KEY (client_name, cost, order_date, order_id)
        WITH CLUSTERING ORDER BY (cost ASC, order_date ASC, order_id ASC)
        AND bloom_filter_fp_chance = 0.01
        AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
        AND comment = ''
        AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
        AND compression = {'chunk_length_in_kb': '64', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
        AND crc_check_chance = 1.0
        AND dclocal_read_repair_chance = 0.1
        AND default_time_to_live = 0
        AND gc_grace_seconds = 864000
        AND max_index_interval = 2048
        AND memtable_flush_period_in_ms = 0
        AND min_index_interval = 128
        AND read_repair_chance = 0.0
        AND speculative_retry = '99PERCENTILE';

In [67]:
# 2. Для замовника виведіть всі його замовлення відсортовані за часом коли вони були зроблені

query = """
SELECT * FROM estore.orders
WHERE client_name = 'Stepan Baran'
"""

rows = session.execute(query)
for row in rows:
    print(row)

Row(client_name='Stepan Baran', order_date=Date(19002), order_id=1, cost=51498, items=[3, 9], order_timestamp=datetime.datetime(2022, 1, 10, 13, 44))
Row(client_name='Stepan Baran', order_date=Date(19005), order_id=2, cost=41999, items=[15], order_timestamp=datetime.datetime(2022, 1, 13, 9, 10))
Row(client_name='Stepan Baran', order_date=Date(19010), order_id=3, cost=41999, items=[12], order_timestamp=datetime.datetime(2022, 1, 18, 18, 35))


In [68]:
# 3. Для замовника знайдіть замовлення з певним товаром

query = """
SELECT * FROM estore.orders
WHERE client_name = 'Stepan Baran'
AND items CONTAINS 3;
"""

rows = session.execute(query)
for row in rows:
    print(row)

Row(client_name='Stepan Baran', order_date=Date(19002), order_id=1, cost=51498, items=[3, 9], order_timestamp=datetime.datetime(2022, 1, 10, 13, 44))


In [71]:
# 4. Для замовника знайдіть замовлення за певний період часу і їх кількість

query = """
SELECT * FROM estore.orders
WHERE client_name = 'Petro Vovk'
AND order_date >= '2022-01-10' AND order_date <= '2022-01-13';
"""

rows = session.execute(query)
for row in rows:
    print(row)

Row(client_name='Petro Vovk', order_date=Date(19003), order_id=4, cost=23999, items=[7], order_timestamp=datetime.datetime(2022, 1, 11, 13, 23))
Row(client_name='Petro Vovk', order_date=Date(19004), order_id=5, cost=35498, items=[10, 11], order_timestamp=datetime.datetime(2022, 1, 12, 19, 10))


In [73]:
# 5. Для кожного замовників визначте суму на яку були зроблені усі його замовлення

query = """
SELECT client_name, SUM(cost) as total_spent FROM estore.orders
GROUP BY client_name;
"""

rows = session.execute(query)
for row in rows:
    print(row)

Row(client_name='Mykola Bez Prykolu', total_spent=121496)
Row(client_name='Petro Vovk', total_spent=90496)
Row(client_name='Stepan Baran', total_spent=135496)


In [74]:
# 6. Для кожного замовників визначте замовлення з максимальною вартістю

query = """
SELECT client_name, MAX(cost) as max_order_value, order_id FROM estore.orders
GROUP BY client_name;
"""

rows = session.execute(query)
for row in rows:
    print(row)

Row(client_name='Mykola Bez Prykolu', max_order_value=75497, order_id=7)
Row(client_name='Petro Vovk', max_order_value=35498, order_id=4)
Row(client_name='Stepan Baran', max_order_value=51498, order_id=1)


In [79]:
# 7. Модифікуйте певне замовлення додавши / видаливши один або кілька товарів при цьому також змінюючи вартість замовлення
query = """
SELECT * FROM estore.orders
WHERE client_name = 'Stepan Baran' AND order_date = '2022-01-10' AND order_id = 1
"""

current_state = session.execute(query).one()
print(current_state.cost)

item_to_add = {'item_id': 5, 'item_price': 21999}

query = f"""
UPDATE estore.orders
SET items = items + [{item_to_add['item_id']}], cost = {current_state.cost + item_to_add['item_price']}
WHERE client_name = 'Stepan Baran' AND order_date = '2022-01-10' AND order_id = 1;
"""
print(query)
session.execute(query)

query = """
SELECT * FROM estore.orders
WHERE client_name = 'Stepan Baran' AND order_date = '2022-01-10' AND order_id = 1
"""

rows = session.execute(query)
for row in rows:
    print(row)

51498

UPDATE estore.orders
SET items = items + [5], cost = 73497
WHERE client_name = 'Stepan Baran' AND order_date = '2022-01-10' AND order_id = 1;

Row(client_name='Stepan Baran', order_date=Date(19002), order_id=1, cost=73497, items=[3, 9, 5], order_timestamp=datetime.datetime(2022, 1, 10, 13, 44))


In [87]:
# 8. Для кожного замовлення виведіть час коли його ціна були занесена в базу (SELECT WRITETIME)

query = """
SELECT WRITETIME (cost), order_id FROM estore.orders;
"""

rows = session.execute(query)
for row in rows:
    print(row)

Row(writetime_cost=1643617905263000, order_id=7)
Row(writetime_cost=1643617905263000, order_id=8)
Row(writetime_cost=1643617902638001, order_id=4)
Row(writetime_cost=1643617902638001, order_id=5)
Row(writetime_cost=1643617902638001, order_id=6)
Row(writetime_cost=1643620062919340, order_id=1)
Row(writetime_cost=1643617900468001, order_id=2)
Row(writetime_cost=1643617900468001, order_id=3)


In [89]:
# 9. Створіть замовлення з певним часом життя (TTL), після якого воно видалиться

query = """
INSERT INTO estore.orders (client_name, order_date, order_id, order_timestamp, cost, items)
VALUES ('Andriy Zaiac', '2022-01-20', 9, '2022-01-20 13:23', 21999, [5]) IF NOT EXISTS
USING TTL 15;
"""

session.execute(query)

query = """
SELECT * FROM estore.orders
WHERE client_name = 'Andriy Zaiac';
"""
row = session.execute(query).one()
if row:
    print(row)
else:
    print('Nothing found!')

time.sleep(16)

row = session.execute(query).one()
if row:
    print(row)
else:
    print('Nothing found!')

Row(client_name='Andriy Zaiac', order_date=Date(19012), order_id=9, cost=21999, items=[5], order_timestamp=datetime.datetime(2022, 1, 20, 13, 23))
Nothing found!


In [91]:
# 10. Поверніть замовлення у форматі JSON

query = """
SELECT json * FROM estore.orders;
"""

rows = session.execute(query)
for row in rows:
    print(row.json)

{"client_name": "Mykola Bez Prykolu", "order_date": "2022-01-15", "order_id": 7, "cost": 75497, "items": [8, 10, 11], "order_timestamp": "2022-01-15 13:23:00.000Z"}
{"client_name": "Mykola Bez Prykolu", "order_date": "2022-01-17", "order_id": 8, "cost": 45999, "items": [16], "order_timestamp": "2022-01-17 19:10:00.000Z"}
{"client_name": "Petro Vovk", "order_date": "2022-01-11", "order_id": 4, "cost": 23999, "items": [7], "order_timestamp": "2022-01-11 13:23:00.000Z"}
{"client_name": "Petro Vovk", "order_date": "2022-01-12", "order_id": 5, "cost": 35498, "items": [10, 11], "order_timestamp": "2022-01-12 19:10:00.000Z"}
{"client_name": "Petro Vovk", "order_date": "2022-01-19", "order_id": 6, "cost": 30999, "items": [13], "order_timestamp": "2022-01-19 14:05:00.000Z"}
{"client_name": "Stepan Baran", "order_date": "2022-01-10", "order_id": 1, "cost": 73497, "items": [3, 9, 5], "order_timestamp": "2022-01-10 13:44:00.000Z"}
{"client_name": "Stepan Baran", "order_date": "2022-01-13", "order_

In [94]:
# 11. Додайте замовлення у форматі JSON

query = """
INSERT INTO estore.orders JSON '{
    "client_name": "Andriy Zaiac",
    "order_date": "2022-01-20",
    "order_id": 9,
    "order_timestamp": "2022-01-20 13:23",
    "cost": 21999,
    "items": [5]
}'
"""

session.execute(query)

query = """
SELECT * FROM estore.orders
WHERE client_name = 'Andriy Zaiac';
"""
row = session.execute(query).one()
print(row)

Row(client_name='Andriy Zaiac', order_date=Date(19012), order_id=9, cost=21999, items=[5], order_timestamp=datetime.datetime(2022, 1, 20, 13, 23))
